In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

In [5]:
# Importing files
season_1 = pd.read_csv("datasets/matches_2017-2018.csv")
season_2 = pd.read_csv("datasets/matches_2018-2019.csv")
season_3 = pd.read_csv("datasets/matches_2019-2020.csv")
season_4 = pd.read_csv("datasets/matches_2020-2021.csv")
season_5 = pd.read_csv("datasets/matches_2021-2022.csv")


In [6]:
# Making an array with all seasons
all_seasons = [season_1, season_2, season_3, season_4, season_5]

# Combining all the files into one dataframe with all 5 seasons
matches = pd.concat(all_seasons)

In [7]:
matches.shape

(3780, 28)

In [8]:
matches['team'].value_counts()

ManchesterCity            189
LeicesterCity             189
Southampton               189
ManchesterUnited          189
WestHamUnited             189
CrystalPalace             189
NewcastleUnited           189
BrightonandHoveAlbion     189
Everton                   189
Arsenal                   189
Chelsea                   189
Liverpool                 189
TottenhamHotspur          189
Burnley                   189
Watford                   151
WolverhamptonWanderers    151
Bournemouth               114
AstonVilla                113
HuddersfieldTown           76
WestBromwichAlbion         76
Fulham                     76
SheffieldUnited            76
LeedsUnited                75
NorwichCity                75
StokeCity                  38
CardiffCity                38
SwanseaCity                38
Brentford                  37
Name: team, dtype: int64

Some Teams have not played all of the 5 seasons, each season contains a total of 38 games. In no particular order, here is how many seasons each team have played:

- Played 1 Seasons
    - StokeCity                  38
    - CardiffCity                38
    - SwanseaCity                38
    - Brentford                  37

- Played 2 Seasons
    - HuddersfieldTown           76
    - WestBromwichAlbion         76
    - Fulham                     76
    - SheffieldUnited            76
    - LeedsUnited                75
    - NorwichCity                75

- Played 3 Seasons
    - Bournemouth               114
    - AstonVilla                113

- Played 4 Seasons
    - Watford                   151
    - WolverhamptonWanderers    151

- Played 5 Seasons

    - ManchesterCity            189
    - LeicesterCity             189
    - Southampton               189
    - ManchesterUnited          189
    - WestHamUnited             189
    - CrystalPalace             189
    - NewcastleUnited           189
    - BrightonandHoveAlbion     189
    - Everton                   189
    - Arsenal                   189
    - Chelsea                   189
    - Liverpool                 189
    - TottenhamHotspur          189
    - Burnley                   189

